<a href="https://colab.research.google.com/github/yashsingla984/stockmarketforecasting/blob/master/Simple_NN_talib_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Set seeds for reproducibility
random.seed(25)
np.random.seed(25)
tf.random.set_seed(25)

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Function to calculate RSI
def calculate_rsi(data, period=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Function to calculate Williams %R
def calculate_williams_r(high, low, close, period=14):
    hh = high.rolling(window=period).max()
    ll = low.rolling(window=period).min()
    williams_r = ((hh - close) / (hh - ll)) * -100
    return williams_r


In [4]:
# Step 1: Fetch historical stock price data for Apple (AAPL)
dataset = yf.download('AAPL', start='2010-01-01', end='2022-01-01')
print(dataset.columns)
# Step 2: Calculate additional features
dataset['H-L'] = dataset['High'] - dataset['Low']
dataset['O-C'] = dataset['Close'] - dataset['Open']
dataset['3day MA'] = dataset['Close'].shift(1).rolling(window=3).mean()
dataset['10day MA'] = dataset['Close'].shift(1).rolling(window=10).mean()
dataset['30day MA'] = dataset['Close'].shift(1).rolling(window=30).mean()
dataset['Std_dev'] = dataset['Close'].rolling(window=5).std()
dataset['RSI'] = calculate_rsi(dataset['Close'])
dataset['Williams %R'] = calculate_williams_r(dataset['High'], dataset['Low'], dataset['Close'])

[*********************100%%**********************]  1 of 1 completed

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')


In [5]:
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout


In [6]:
dataset['Price_Rise'] = np.where(dataset['Close'].shift(-1) > dataset['Close'], 1, 0)
dataset = dataset.dropna()
print(dataset.head())
X = dataset.iloc[:, 4:-1]
y = dataset.iloc[:, -1]

split = int(len(dataset)*0.9)
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]

                Open      High       Low     Close  Adj Close     Volume  \
Date                                                                       
2010-02-17  7.292500  7.296786  7.173571  7.233929   6.124240  436396800   
2010-02-18  7.201071  7.281786  7.175714  7.247500   6.135731  422825200   
2010-02-19  7.209286  7.257143  7.182500  7.202500   6.097632  415469600   
2010-02-22  7.226429  7.232143  7.113929  7.157857   6.059838  390563600   
2010-02-23  7.142857  7.190357  6.989643  7.037857   5.958245  575094800   

                 H-L       O-C   3day MA  10day MA  30day MA   Std_dev  \
Date                                                                     
2010-02-17  0.123215 -0.058571  7.172024  7.037429  7.275952  0.118165   
2010-02-18  0.106072  0.046429  7.218215  7.061321  7.262310  0.071392   
2010-02-19  0.074643 -0.006786  7.248572  7.074536  7.248679  0.042587   
2010-02-22  0.118214 -0.068572  7.227976  7.108893  7.237607  0.042048   
2010-02-23  0.200714 -0

In [7]:
sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

In [8]:
classifier = Sequential()
classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu', input_dim = X.shape[1]))
classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])

In [9]:
classifier.fit(X_train, y_train, batch_size = 16, epochs = 10)

Epoch 1/10
169/169 [==============================] - 7s 10ms/step - loss: 0.4701 - accuracy: 0.5299
Epoch 2/10
169/169 [==============================] - 1s 3ms/step - loss: 0.4701 - accuracy: 0.5299
Epoch 3/10
169/169 [==============================] - 1s 4ms/step - loss: 0.4701 - accuracy: 0.5299
Epoch 4/10
169/169 [==============================] - 1s 4ms/step - loss: 0.4701 - accuracy: 0.5299
Epoch 5/10
169/169 [==============================] - 1s 4ms/step - loss: 0.4701 - accuracy: 0.5299
Epoch 6/10
169/169 [==============================] - 1s 4ms/step - loss: 0.4701 - accuracy: 0.5299
Epoch 7/10
169/169 [==============================] - 1s 3ms/step - loss: 0.4701 - accuracy: 0.5299
Epoch 8/10
169/169 [==============================] - 1s 3ms/step - loss: 0.4701 - accuracy: 0.5299
Epoch 9/10
169/169 [==============================] - 1s 4ms/step - loss: 0.4701 - accuracy: 0.5299
Epoch 10/10
169/169 [==============================] - 1s 3ms/step - loss: 0.4701 - accuracy: 0.529

In [10]:
scores = classifier.evaluate(X_test, y_test, verbose=1) # scores has [loss] and [accuracy]
print(scores)
#Print out the error rate
print('Simple NN Accuracy: %.2f%%'% (scores[1]*100))
print("Simple NN Error:    %.2f%%"% (100-scores[1]*100))

10/10 [==============================] - 0s 3ms/step - loss: 0.4700 - accuracy: 0.5300
[0.4699999988079071, 0.5299999713897705]
Simple NN Accuracy: 53.00%
Simple NN Error:    47.00%


In [11]:
# import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix, accuracy_score

# y_pred = classifier.predict(X_test)
# y_pred_binary = (y_pred > 0.5)  # Convert probabilities to binary predictions

# # Step 4: Calculate accuracy
# accuracy = accuracy_score(y_test, y_pred_binary)
# print("Accuracy:", accuracy)

# # Step 5: Plot the confusion matrix
# cm = confusion_matrix(y_test, y_pred_binary)
# plt.figure(figsize=(8, 6))
# plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
# plt.title('Confusion Matrix')
# plt.colorbar()
# plt.xlabel('Predicted Label')
# plt.ylabel('True Label')
# plt.show()

In [12]:
# # Make predictions on the test set
# predictions = classifier.predict(X_test)

# # Plot actual vs predicted values
# plt.figure(figsize=(10, 6))
# plt.scatter(y_test, predictions)
# plt.title('Actual vs Predicted Values')
# plt.xlabel('Actual Values')
# plt.ylabel('Predicted Values')
# plt.show()